In [1]:
from dotenv import load_dotenv
from onnxruntime.transformers.models.stable_diffusion.benchmark import example_prompts
from torch.utils.flop_counter import suffixes

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


# FewShotPromptTemplate

In [3]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 질의 내용
question = "대한민국의 수도는 어디야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

대한민국의 수도는 서울입니다.

In [4]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [5]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)
print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [7]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google 이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google 이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [8]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 언제 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에는 Bill Gates가 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 42세였습니다.  
최종 답변은: 42세

In [10]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"]
)

# chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream({"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"})
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 몇 년에 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에는 Bill Gates가 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 42세였습니다.  
최종 답변은: 42세

# Example Selector

In [13]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 샏성하는 임베딩 클래스
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사요되는 VectorStore 클랙스
    Chroma,
    # 이것은 생성할 예시의 수
    k=1,
)

# 입력과 가장 유사한 예시를 선택
selected_examples = example_selector.select_examples({"question": question})

question = "Google이 창립된 연도에 bill Gates의 나이는 몇 살인가요?"
print(f"입력이 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f"question:\n{example['question']}")
    print(f"answer:\n{example['answer']}")

입력이 가장 유사한 예시:
Google이 창립된 연도에 bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [15]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [21]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | llm

In [22]:
# 결과 출력
answer = chain.stream({"question" : "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"})
stream_response(answer)

Google은 1998년에 창립되었습니다. Bill Gates는 1955년 10월 28일에 태어났으므로, 1998년에는 42세였습니다. 최종 답변은: 42세.

# FewShotChatMessagePromptTemplate

In [23]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]


In [24]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}")
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    chroma,
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [25]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [26]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpfuil assistant."),
        few_shot_prompt,
        ("human", "{instruction}:\n{input}")
    ]
)

In [27]:
# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사 제품 개발 팀 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤 계획 수립.

2. 진행 상황 업데이트
   - **최현수 (프로젝트 매니저)**: 전체 프로젝트 일정 및 현재 진행 상황에 대한 개요 제공. 
   - **황지연 (주요 개발자)**: 개발 진행 상황 보고. 주요 기능 구현 완료 및 버그 수정 작업 진행 중.
   - **김태영 (UI/UX 디자이너)**: 디자인 프로토타입 완성 및 사용자 피드백 수집 결과 공유. UI 개선 사항 논의.

3. 마일스톤 계획
   - 팀원들은 다음 주까지의 목표 설정.
   - 각 팀원은 자신의 작업 영역에 대한 구체적인 목표를 설정하고, 필요한 자원 및 지원 요청.

4. 종합 논의
   - 팀원들 간의 의견 교환 및 문제 해결 방안 논의.
   - 다음 주 회의 일정 및 준비 사항 확인.

5. 마무리
   - 최현수 프로젝트 매니저가 회의를 마무리하며, 각 팀원에게 다음 주 목표 달성을 위한 협력 요청.
   - 회의록 작성 및 배포는 최현수 담당.

# Example Selector 의 유사도 검색 문제 해결

In [28]:
question = {
    "instruction": "회의록을 작성해 주세요."
}

example_selector.select_examples(question)

[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [29]:
# 커스텀 하지 않은 기본 예제 선택기를 사용했을 때 결과
example_selector.select_examples({"instruction": "다음 문장을 요약해 주세요."})

[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [30]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "다음 문장을 교정 작성해 주세요."})

[{'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.'}]

In [31]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}")
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,
    example_prompt=example_prompt,
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpfuil assistant."),
        custom_fewshot_prompt,
        ("human", "{instruction}:\n{input}")
    ]
)

In [32]:
# chain 을 생성합니다.
chain = custom_prompt | llm

In [33]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


회의록: ABC 기술 회사 제품 개발 팀 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤 계획 수립.

2. 진행 상황 업데이트
   - **최현수 (프로젝트 매니저)**: 전체 프로젝트 일정 및 현재 진행 상황에 대한 개요 제공.
   - **황지연 (주요 개발자)**: 개발 진행 상황 및 기술적 이슈에 대한 업데이트. 현재 기능 구현 상태와 해결된 문제들에 대해 설명.
   - **김태영 (UI/UX 디자이너)**: 디자인 프로토타입 및 사용자 피드백에 대한 업데이트. 사용자 경험 개선을 위한 제안 사항 공유.

3. 목표 설정
   - 팀원들은 다음 주까지의 목표를 설정하고 각자의 작업 계획을 논의.
   - 마일스톤에 대한 일정 조정 및 필요한 자원 확인.

4. 종합 논의
   - 팀원들 간의 의견 교환 및 추가적인 아이디어 논의.
   - 프로젝트 진행에 필요한 지원 사항 및 협력 방안 논의.

5. 마무리
   - 다음 주 회의 일정 확정.
   - 회의록 작성 및 배포는 최현수 담당.

In [34]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


문서 요약: 2023년 글로벌 경제 전망 보고서

- **현재 상태**: 세계 경제의 현황을 분석하고, 주요 국가들의 경제 성장률을 평가.
- **경제 성장률**: 주요 국가들의 성장률 변화와 그에 따른 경제적 영향에 대한 데이터 제공.
- **글로벌 무역 동향**: 최근의 무역 흐름과 변화, 그리고 이로 인한 경제적 결과를 설명.
- **경제 예측**: 다가오는 해에 대한 경제 전망과 예상되는 주요 트렌드 제시.
- **영향 요인**: 경제적, 정치적, 환경적 요인들이 세계 경제에 미치는 영향을 심층적으로 분석.

이 보고서는 2023년 글로벌 경제의 전반적인 전망을 제시하며, 다양한 요인들이 경제에 미치는 복합적인 영향을 이해하는 데 도움을 줍니다.

In [35]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


회사는 올해 매출이 증가할 것으로 예상하고 있으며, 새로운 전략이 잘 작동하고 있다.